In [1]:
#tensorいじりのテストを行う
import torch
import torch.nn
x = torch.randn(10,10,10)
print(torch.cuda.is_available())
from audio_to_multiple_pose_gan.dataset import generate_batch, get_processor
from audio_to_multiple_pose_gan.config import get_config
import warnings
warnings.simplefilter('ignore')

True


In [2]:
import pandas as pd
from common.consts import AUDIO_SHAPE
batch_size = 32
train_csv: str = "~/Desktop/AI-Experiments/speechandtext2gesture/Gestures/train.csv"
#train_csv: str = "~/AI-Experiments/speechandtext2gesture/Gestures/train.csv"
#df = pd.read_csv(self.args.train_csv)
#cfg = get_config(self.args.config)
df = pd.read_csv(train_csv)
#cfg = get_config(config)
cfg: dict = {"processor": "audio_to_pose", "input_shape": [None, AUDIO_SHAPE]}
process_row, decode_pose = get_processor(cfg)

In [3]:
from common.consts import AUDIO_SHAPE
configs = {
    "audio_to_pose": {"num_keypoints": 98, "processor": "audio_to_pose", "flatten": False, "input_shape": [None, AUDIO_SHAPE]},
    "audio_to_pose_inference": {"num_keypoints": 98, "processor": "audio_to_pose_inference", "flatten": False, "input_shape": [None, AUDIO_SHAPE]}
}
cfg: dict = {"processor": "audio_to_pose", "input_shape": [None, AUDIO_SHAPE]}
process_row, decode_pose = get_processor(cfg)

In [4]:
row = df.sample(n=1).iloc[0]
X, Y = generate_batch(df, process_row, batch_size)

In [5]:
Y.shape

(32, 64, 98)

In [6]:
X.shape

(32, 67267)

In [7]:
df.sample(n=1).iloc[0]

dataset                                                    train
start                                            00:02:02.655989
end                                              00:02:06.860194
interval_id                                                 1524
pose_fn        Gestures/shelly/train/npz/1524-00:02:02.655989...
audio_fn       Gestures/shelly/train/audio/1524-00:01:54.9816...
video_fn       14._What_matters_cont._The_nature_of_death_Par...
speaker                                                   shelly
Name: 56858, dtype: object

In [8]:
from audio_to_multiple_pose_gan.static_model_factory import *
a2p = Audio2Pose(1, 98).to(device)
a2p(torch.tensor(X).float().to(device)).shape

torch.Size([32, 64, 98])

In [9]:
Y.shape

(32, 64, 98)

In [10]:
d_patchgan = D_patchgan(in_channels=64)

In [11]:
Y.shape

(32, 64, 98)

In [12]:
Y.shape

(32, 64, 98)

In [13]:
from audio_to_multiple_pose_gan.torch_layers import *
to_motion_delta(Y).shape

(32, 63, 98)

In [14]:
Y = torch.tensor(Y).float().to(device)

In [15]:
a2p = Audio2PoseGANS(1, 98).to(device)
pred_Y = a2p(torch.tensor(X).float().to(device))

torch.Size([32, 256, 64])


In [16]:
keypoints_regloss(Y, pred_Y, regloss_type="l1")

tensor(0.8180, device='cuda:0', grad_fn=<MeanBackward0>)

In [24]:
from audio_to_multiple_pose_gan.config import get_config
from audio_to_multiple_pose_gan.dataset import generate_batch, get_processor
#get_modelの実装は任せる
from audio_to_multiple_pose_gan.static_model_factory import Audio2PoseGANS, D_patchgan
from audio_to_multiple_pose_gan.static_model_factory import Audio2PoseGANSTransformer
from audio_to_multiple_pose_gan.torch_layers import to_motion_delta, keypoints_to_train, keypoints_regloss
from common.audio_lib import save_audio_sample
from common.audio_repr import raw_repr
from common.consts import RIGHT_BODY_KEYPOINTS, LEFT_BODY_KEYPOINTS, LEFT_HAND_KEYPOINTS, \
    RIGHT_HAND_KEYPOINTS, POSE_SAMPLE_SHAPE, G_SCOPE, D_SCOPE, SR, FRAMES_PER_SAMPLE, AUDIO_SHAPE
from common.evaluation import compute_pck
from common.pose_logic_lib import translate_keypoints, get_sample_output_by_config
from common.pose_plot_lib import save_side_by_side_video, save_video_from_audio_video
from torch import optim
import torch
import pandas as pd
import argparse
from torch import nn

def get_training_keypoints():
    training_keypoints = []
    training_keypoints.extend(RIGHT_BODY_KEYPOINTS)
    training_keypoints.extend(LEFT_BODY_KEYPOINTS)
    for i in range(5):
        training_keypoints.extend(RIGHT_HAND_KEYPOINTS(i))
        training_keypoints.extend(LEFT_HAND_KEYPOINTS(i))
    training_keypoints = sorted(list(set(training_keypoints)))
    return training_keypoints

In [25]:
poses = decode_pose(Y.cpu().numpy()[0], shift = [0,0], speaker="shelly")

In [26]:
G_model = Audio2PoseGANSTransformer(1, 98).to(device)

In [27]:
state_dict = torch.load("params/G_model_last.pth")

In [28]:
from collections import OrderedDict
param_dict = OrderedDict()
for key in state_dict:
    param_dict[key.replace("_orig_mod.","")] = state_dict[key]

In [30]:
G_model(torch.tensor(X).float().to(device))

tensor([[[ 4.1996e-04,  1.1409e-02,  8.9636e-01,  ...,  2.2891e-01,
           2.2697e-01,  2.2480e-01],
         [ 1.1315e-03, -3.7908e-02,  9.9225e-01,  ...,  1.8634e-01,
           1.6393e-01,  1.7367e-01],
         [ 2.4469e-04, -5.4689e-02,  1.0447e+00,  ...,  1.0025e-01,
           9.1471e-02,  1.0490e-01],
         ...,
         [ 3.0918e-04, -6.3821e-01, -5.5820e-01,  ...,  7.3165e-01,
           7.6042e-01,  7.6217e-01],
         [-4.5094e-05, -6.7813e-01, -5.4253e-01,  ...,  7.2746e-01,
           7.4406e-01,  7.4936e-01],
         [ 3.3324e-04, -6.8983e-01, -5.2097e-01,  ...,  6.7416e-01,
           6.8064e-01,  6.8525e-01]],

        [[ 6.6774e-04, -8.4116e-02,  2.2074e-01,  ..., -9.1132e-01,
          -9.1675e-01, -9.2759e-01],
         [ 1.1174e-03, -1.8906e-01,  1.9895e-01,  ..., -8.9615e-01,
          -9.2028e-01, -9.2478e-01],
         [ 9.4345e-04, -1.7329e-01,  3.0695e-01,  ..., -8.8341e-01,
          -9.1654e-01, -9.1810e-01],
         ...,
         [ 7.1185e-04, -2